# Calculated Win Percentage VS Actual Win Percentage

Data obtain from: https://www.kaggle.com/datasets/timschutzyang/dataset1?resource=download

**Goal:** The goal of this project is to identify the Error in Winning Percentage in each year using the Pythagorean Theorem of baseball. $$ (Runs Scored)^2  /  (Runs Scored)^2 + (Runs Allowed)^2 $$

A more accurate version of the formula uses an exponent of 1.81 rather than 2. Our dataset we will be importing uses the more accurate formula.

## Import and clean my data

The first thing I want to do is import my data and get it ready to be used. I import my library, bring in my CSV file, and test to make sure it is gathered properly.

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# import csv
df = pd.read_csv('baseballdata.csv')

# check data frame
df.head()

,Unnamed: 0,Rk,Year,Tm,Lg,G,W,L,Ties,W.L.,...,R,RA,Attendance,BatAge,PAge,X.Bat,X.P,Top.Player,Managers,current
0,1,1,2016,Arizona Diamondbacks,NL West,162,69,93,0,0.426,...,752,890,"2,036,216",26.7,26.4,50,29,J.Segura (5.7),C.Hale (69-93),Arizona Diamondbacks
1,2,2,2015,Arizona Diamondbacks,NL West,162,79,83,0,0.488,...,720,713,"2,080,145",26.6,27.1,50,27,P.Goldschmidt (8.8),C.Hale (79-83),Arizona Diamondbacks
2,3,3,2014,Arizona Diamondbacks,NL West,162,64,98,0,0.395,...,615,742,"2,073,730",27.6,28.0,52,25,P.Goldschmidt (4.5),K.Gibson (63-96) and A.Trammell (1-2),Arizona Diamondbacks
3,4,4,2013,Arizona Diamondbacks,NL West,162,81,81,0,0.500,...,685,695,"2,134,895",28.1,27.6,44,23,P.Goldschmidt (7.1),K.Gibson (81-81),Arizona Diamondbacks
4,5,5,2012,Arizona Diamondbacks,NL West,162,81,81,0,0.500,...,734,688,"2,177,617",28.3,27.4,48,23,A.Hill (5.0),K.Gibson (81-81),Arizona Diamondbacks


With my data imported I can drop the columns I don't need. In this case I am only interested in the following rows: `Year`, `Tm`, `W`, `L`, `W.L.`, `pythW.L.`, `R`, `RA`.

In [3]:
# get the name of each column
df.columns

Index(['Unnamed: 0', 'Rk', 'Year', 'Tm', 'Lg', 'G', 'W', 'L', 'Ties', 'W.L.',
       'pythW.L.', 'Finish', 'GB', 'Playoffs', 'R', 'RA', 'Attendance',
       'BatAge', 'PAge', 'X.Bat', 'X.P', 'Top.Player', 'Managers', 'current'],
      dtype='object')

In [4]:
# create a new dataframe with only the columns that I need
newdf = df[['Tm', 'W', 'L', 'W.L.', 'pythW.L.', 'R', 'RA']]

#newdf.head()

In [6]:
# rename columns to preference
new_names = {'Tm': 'Team', 'W': 'Wins', 'L': 'Losses', 'W.L.': 'ActualWinPct',
             'pythW.L.': 'PythWinPct', 'R': 'Runs', 'RA': 'RunsAllowed'}

newdf = newdf.rename(columns=new_names)

newdf.head()

,Team,Wins,Losses,ActualWinPct,PythWinPct,Runs,RunsAllowed
0,Arizona Diamondbacks,69,93,0.426,0.424,752,890
1,Arizona Diamondbacks,79,83,0.488,0.504,720,713
2,Arizona Diamondbacks,64,98,0.395,0.415,615,742
3,Arizona Diamondbacks,81,81,0.500,0.493,685,695
4,Arizona Diamondbacks,81,81,0.500,0.530,734,688


In [7]:
#check the data types of my columns
newdf.dtypes

Team             object
Wins              int64
Losses            int64
ActualWinPct    float64
PythWinPct      float64
Runs              int64
RunsAllowed       int64
dtype: object

In [9]:
# create a new column showing the error in winning percentages
newdf['WinPctError'] = newdf.ActualWinPct - newdf.PythWinPct

newdf.head()

,Team,Wins,Losses,ActualWinPct,PythWinPct,Runs,RunsAllowed,WinPctError
0,Arizona Diamondbacks,69,93,0.426,0.424,752,890,0.002
1,Arizona Diamondbacks,79,83,0.488,0.504,720,713,-0.016
2,Arizona Diamondbacks,64,98,0.395,0.415,615,742,-0.020
3,Arizona Diamondbacks,81,81,0.500,0.493,685,695,0.007
4,Arizona Diamondbacks,81,81,0.500,0.530,734,688,-0.030


In [13]:
# get a breakdown of the column results
newdf.WinPctError.describe()

count    2594.000000
mean       -0.000069
std         0.026391
min        -0.090000
25%        -0.018000
50%         0.000000
75%         0.018000
max         0.091000
Name: WinPctError, dtype: float64

I am interested to know the average that the calculation was off by. The reason we can't just take the `mean` of `WinPctError` is because having both positive and negative values will cause the values to cancel out, not give us the average. For example, if the error was off by 3% for one year and -3% for another year the mean would result in 0. But this is not an accurate calculation. It would be true to say that the average error between those two numbers is still 3%.

In [15]:
# create new column for the absolute value of the WinPctError
newdf['ErrorAbs'] = abs(newdf.WinPctError)

newdf.head()

,Team,Wins,Losses,ActualWinPct,PythWinPct,Runs,RunsAllowed,WinPctError,ErrorAbs
0,Arizona Diamondbacks,69,93,0.426,0.424,752,890,0.002,0.002
1,Arizona Diamondbacks,79,83,0.488,0.504,720,713,-0.016,0.016
2,Arizona Diamondbacks,64,98,0.395,0.415,615,742,-0.020,0.020
3,Arizona Diamondbacks,81,81,0.500,0.493,685,695,0.007,0.007
4,Arizona Diamondbacks,81,81,0.500,0.530,734,688,-0.030,0.030


Now that I have a column for the absolute value of the `WinPctError` we can find the `mean`.

In [17]:
# find the mean of the error
newdf.ErrorAbs.mean()

0.02108172706245201

With a result above we can conclude that the Pythagorean Therom of baseball had average error of 2%